# IMPORT

In [87]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

llm_model = ChatGoogleGenerativeAI(model="google-1.5-flash",temperature=0.7) 

# PROMPT

In [88]:
from langchain.prompts import ChatPromptTemplate
from langchain.pydantic_v1 import Field , BaseModel
from typing import Optional
from datetime import date
from langchain_core.output_parsers import JsonOutputParser

class TaskRequest(BaseModel):
    action: str = Field(..., description="Action type: add, update, delete, list")
    title: Optional[str] = Field(None, description="The title of the task (required for add/update/delete)")
    task_date: Optional[date] = Field(None, description="The date of the task (optional, defaults to today)")


task_request_parser = JsonOutputParser(pydantic_object=TaskRequest)

DB_SCHEMA_PROMPT = """
The database has two tables:

Table: users
- email (PRIMARY KEY, VARCHAR)
- password (VARCHAR)
- url_hash (VARCHAR)

Table: tasks
- id (PRIMARY KEY, AUTO_INCREMENT, INT)
- task_date (DATE)
- title (VARCHAR)
- user_email (VARCHAR, FOREIGN KEY → users.email)
"""


MAIN_PROMPT = ChatPromptTemplate.from_messages([("system",'''You are a SQL assistant.
Only output valid MySQL queries, nothing else.
Return the query in 'JSON object' with 'answer' as key.
{format_instructions} 

Use the schema below:
{DB_SCHEMA_PROMPT}

Rules:
1. You will receive a structured JSON request following the TaskRequest schema.
2. User email is never provided. Always use '{{session_user_email}}' for user_email.
3. If task_date is missing in ADD, use CURRENT_DATE.
4. For UPDATE and DELETE, require both id and user_email.
5. Always return raw SQL only (no explanations, no markdown).'''),
                                                ("human","{user_input}")]).partial(format_instructions=task_request_parser.get_format_instructions(),DB_SCHEMA_PROMPT=DB_SCHEMA_PROMPT)

In [89]:

llm_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")


chain = MAIN_PROMPT | llm_model | task_request_parser

# Invoke without config
request = chain.invoke({"user_input": "add breakfast at 12/12/2025"})
print(request)


KeyboardInterrupt: 

# LOAD DATE

In [ ]:
from langchain_community.document_loaders import JSONLoader , TextLoader

# DATA = JSONLoader(file_path="D:\\NameKart\\React + SpringBoot\\ChatBot\\Resource\\Credits.json", jq_schema=".[]", text_content=False)
# documents = DATA.load()
# documents[0].page_content

DATA = TextLoader(file_path="D:\\NameKart\\React + SpringBoot\\ChatBot\\Resource\\data.txt")
documents = DATA.load()
documents


In [ ]:
from langchain_text_splitters import CharacterTextSplitter , RecursiveJsonSplitter

text_splitter = CharacterTextSplitter(separator="\n\n",chunk_size=100,chunk_overlap=50,length_function=len,is_separator_regex=False)

In [ ]:
fine_data = text_splitter.create_documents([documents[0].page_content])

In [ ]:
fine_data

# RECURSIVE TEXTSPLITTER

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=100)


In [ ]:
text = text_splitter.split_text(documents[0].page_content)
for i in text:
    print(i)    

# Embedding

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedd_model = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
embedded = embedd_model.embed_documents(text)

In [ ]:
embedded

In [ ]:
len(embedded[3])

3072

# DATABASE STORE

In [ ]:

from langchain_chroma import Chroma
from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
 
loaded_data = TextLoader(file_path="D:/NameKart/React + SpringBoot/ChatBot/Resource/data.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000 , chunk_overlap=0)
chunk_text = text_splitter.split_documents(loaded_data)

In [ ]:
len(chunk_text)

2

In [ ]:
from langchain_community.vectorstores import FAISS
embedding_model = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
vector_db = FAISS.from_documents(chunk_text , embedding=embedding_model)

In [ ]:
question = "is there any name sujith?"
responce = vector_db.as_retriever( search_kwargs={"k":2})
request = responce.invoke(question)
print(request)

KeyboardInterrupt: 